In [ ]:
import tempfile
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml
from spf.dataset.spf_dataset import v5spfdataset

n = 1025
noise = 1.0
nthetas = 65
orbits = 4

tmpdir = tempfile.TemporaryDirectory()
tmpdirname = tmpdir.name
ds_fn = f"{tmpdirname}/sample_dataset_for_ekf_n{n}_noise{noise}"

create_fake_dataset(
    filename=ds_fn, yaml_config_str=fake_yaml, n=n, noise=noise, orbits=orbits
)
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    skip_signal_matrix=True,
)

In [ ]:
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    skip_signal_matrix=True,
    gpu=True,
)

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset

ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
# ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"


nthetas = 65
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache="/home/mouse9911/precompute_cache_chunk16",
    paired=True,
    skip_signal_matrix=True,
)

In [ ]:
import os


output_prefix = "./" + os.path.basename(ds_fn) + "_"

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

for rx_idx in [0, 1]:
    ax[rx_idx].scatter(
        range(len(ds)),
        ds.mean_phase[f"r{rx_idx}"],
        label=f"radio{rx_idx} est phi",
        s=1.0,
        color="red",
    )
    ax[rx_idx].plot(ds.ground_truth_phis[rx_idx], label="perfect phi", color="blue")
    ax[rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(len(ds))],
        label=f"radio{rx_idx} gt theta",
        color="green",
    )
    ax[rx_idx].set_title(f"Radio {rx_idx}")
    ax[rx_idx].set_xlabel("Time step")
    ax[rx_idx].set_ylabel("tehta/phi")
    ax[rx_idx].legend()
    ax[rx_idx].axhline(y=0, color="r", linestyle="-")
fig.suptitle("Phase(phi) recovered from radios after segmentation")
fig.savefig(f"{output_prefix}_raw_signal.png")

In [ ]:
import numpy as np

"""
x = [ theta dtheta/dt ]
z = [ phi ]

F = [ [ 1 dt ],
      [ 0  1 ]]

h(x) = sin(x[0]) * (d * 2 * pi / wavelength )

H(x) = [ dh/dx_1 , dh/dx_2 ] = cos(x[0]) * (d * 2 * pi / wavelength )

"""

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def hx(x):
    return np.array([np.sin(x[0, 0]) * antenna_spacing * 2 * np.pi / wavelength])


def HJacobian_at(x):
    """compute Jacobian of H matrix at x"""
    r = np.array([[np.cos(x[0, 0]) * antenna_spacing * 2 * np.pi / wavelength, 0]])
    # if np.abs(r[0,0])<0.1:
    #    r=r*0+np.sign(r)*0.1
    return r

In [ ]:
from filterpy.common import Q_discrete_white_noise

Q_discrete_white_noise(2, dt=1.0, var=0.3)

In [ ]:
import numpy as np

from spf.rf import pi_norm


def residual(a, b):
    # return pi_norm(a - b)
    y = a - b
    y = y % (2 * np.pi)  # force in range [0, 2 pi)
    if y > np.pi:  # move to [-pi, pi)
        y -= 2 * np.pi
    return y


x = 1.9 * np.pi
y = -1.9 * np.pi
for x in np.linspace(-np.pi, np.pi, 32):
    for y in np.linspace(-np.pi, np.pi, 32):
        assert np.isclose(residual(x, y), pi_norm(x - y), atol=0.01)

In [ ]:
len(ds)

In [ ]:
from functools import cache
from filterpy.kalman import ExtendedKalmanFilter

from spf.rf import pi_norm_halfpi, reduce_theta_to_positive_y


@cache
def Q_discrete_white_noise_cached(**kwargs):
    return Q_discrete_white_noise(**kwargs)


@cache
def F_cached(dt):
    return np.eye(2) + np.array([[0, 1], [0, 0]]) * dt


class SingleRadioConstantTheta(ExtendedKalmanFilter):

    def __init__(self, phi_std=0.5, p=5, **kwargs):
        super().__init__(**kwargs)
        self.R *= phi_std**2
        self.P *= p  # initialized as identity?

    def R_at_x(self):
        return 2.5 * np.exp(-((abs(pi_norm(self.x[0, 0])) - np.pi / 2) ** 2))

    def fix_x(self):
        self.x[0] = pi_norm_halfpi(self.x[0])
        return
        while np.abs(self.x[0]) > np.pi / 2:
            if self.debug:
                print("FLIPPING", self.x)
            self.x[0] = np.sign(self.x[0]) * np.pi - self.x[0]
            self.x[1] *= -1
            if self.debug:
                print("FLIPPED", self.x)

    ###
    #    x[0] = theta , ?
    #    x[1] = theta dot , ? / s
    ###
    def predict(self, dt, q_var):
        self.F = F_cached(dt)
        self.Q = Q_discrete_white_noise_cached(
            dim=2, dt=dt, var=q_var
        )  # TODO Cache this
        ### predict self.x
        self.x = np.dot(self.F, self.x)
        self.fix_x()
        ###

        # update covar
        self.P = np.dot(self.F, self.P).dot(self.F.T) + self.Q

        # do some house keeping
        self.x_prior = np.copy(self.x)
        self.P_prior = np.copy(self.P)

    def update(self, z):
        if self.debug:
            print("DEBUG")
        # find a good z!
        # print(self.R.shape)
        # if z is > 0 consider z and z-2*pi
        # if z is < 0 consider z and z+2*pi
        # if abs(self.x[0]) > 1.5:
        #     predict_phi = hx(self.x)[0]
        #     candidates = np.array([z, z - np.sign(z) * np.pi])
        #     closer_candidate = np.argmax(-np.abs(predict_phi - candidates))
        #     z = candidates[closer_candidate]
        # self.R = np.eye(2) * self.R_at_x(self.x)
        # print(closer_candidate, np.abs(predict_phi - candidates))
        # r = np.array([[self.R_at_x()]]) * 30
        r = (np.array([[self.R_at_x()]]) ** 2) * 5
        if self.debug:
            print("R", r)
            print("P", self.P)
        super().update(
            # np.array([[z]]),
            np.array([[z]]),
            HJacobian_at,
            hx,
            residual=residual,
            R=r,
        )
        # self.x[0] = reduce_theta_to_positive_y(self.x[0])
        self.fix_x()

In [ ]:
def trajectory_for_phi(rx_idx, ds):
    ekf = SingleRadioConstantTheta(dim_x=2, dim_z=1, phi_std=5.0, p=5)
    ekf.x = np.array([[ds[rx_idx][0]["ground_truth_theta"].item()], [0]])
    traj = []
    n = 48 + len(ds)
    jacobian = []
    zs = []
    for idx in range(min(n, len(ds))):
        # ekf.predict(dt=0.1, q_var=0.3) # simulated data
        z = ds[idx][rx_idx]["mean_phase_segmentation"]
        debug = idx >= n - 8
        ekf.debug = debug
        if debug:
            print(idx, "X", ekf.x)
            print(idx, "z", z)
        # ekf.predict(dt=1.0, q_var=0.003)
        ekf.predict(dt=0.05, q_var=0.3)
        if debug:
            print(idx, "X^'", ekf.x)
        ekf.update(z)
        zs.append(z)
        jacobian.append(HJacobian_at(ekf.x)[0, 0])
        if debug:
            print(idx, "X^", ekf.x)
        traj.append({"theta": ekf.x[0, 0], "P_theta": ekf.P[0, 0]})
    return traj, jacobian, zs

In [ ]:
trajectory, jacobian = trajectory_for_phi(0, ds)

In [ ]:
plt.imshow(ds[450][0]["windowed_beamformer"].mean(axis=0, keepdims=True))

In [ ]:
ds.mean_phase[f"r{rx_idx}"][450]

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 2, figsize=(10, 15))

for rx_idx in [0]:  # [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    trajectory, jacobian, zs = trajectory_for_phi(rx_idx, ds)
    jacobian = np.array(jacobian)
    zs = np.array(zs)
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[0, rx_idx].plot(jacobian, label="jacobian")
    ax[0, rx_idx].plot(zs, label="zs")
    ax[0, rx_idx].plot(np.clip(zs / jacobian, a_min=-5, a_max=5), label="zs/j")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
        label=f"r{rx_idx} gt theta",
    )
    reduced_gt_theta = np.array(
        [
            reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
            for idx in range(min(n, len(ds)))
        ]
    )
    ax[1, rx_idx].plot(
        reduced_gt_theta,
        label=f"r{rx_idx} reduced gt theta",
    )

    xs = np.array([x["theta"] for x in trajectory])
    stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
    zscores = (xs - reduced_gt_theta) / stds

    ax[1, rx_idx].plot(xs, label="EKF-x", color="orange")
    ax[1, rx_idx].fill_between(
        np.arange(xs.shape[0]),
        xs - stds,
        xs + stds,
        label="EKF-std",
        color="orange",
        alpha=0.2,
    )

    ax[0, rx_idx].set_ylabel("radio phi")

    ax[0, rx_idx].legend()
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].legend()
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")

    ax[2, rx_idx].hist(zscores.reshape(-1), bins=25)
fig.suptitle("Single ladies (radios) EKF")
fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter

from filterpy.common import Q_discrete_white_noise

from spf.rf import pi_norm, reduce_theta_to_positive_y


def residual(a, b):
    # return pi_norm(a - b)
    y = a - b
    y = y % (2 * np.pi)  # force in range [0, 2 pi)
    if y > np.pi:  # move to [-pi, pi)
        y -= 2 * np.pi
    return y


def trajectory_for_phi(rx_idx, ds):
    rk = ExtendedKalmanFilter(dim_x=2, dim_z=1)
    # initialize with first ground truth state
    y_rad = ds[rx_idx][0]["ground_truth_theta"].item()
    # y_rad_reduced=reduce_theta_to_positive_y(y_rad)
    rk.x = np.array([[y_rad], [0]])

    # dt = 0.1  # 1.0  # 0.1
    dt = 0.1  # 1.0  # 0.1
    rk.F = np.eye(2) + np.array([[0, 1], [0, 0]]) * dt

    phi_std = 0.5
    rk.R = phi_std**2  # np.diag([phi_std**2])
    # TODO R:
    # Use gt state x to get a good R, then worry about how to use estimated x
    # Motion model , f , wrap around
    # make class for motion model and measurement model
    # Plot how x and p evolve
    rk.Q = Q_discrete_white_noise(2, dt=dt, var=0.3)
    rk.P *= 5  # initialized as identity?

    traj = []
    for idx in range(len(ds)):
        rk.update(
            np.array([ds[idx][rx_idx]["mean_phase_segmentation"]]),
            HJacobian_at,
            hx,
            residual=residual,
        )
        traj.append(rk.x[0, 0])
        rk.predict()
        rk.x = pi_norm(rk.x)  # this should not be for theta dot ! TODO BUG!
    return np.array(traj)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

for rx_idx in [0, 1]:
    ax[0, rx_idx].scatter(
        range(ds.mean_phase[f"r{rx_idx}"].shape[0]),
        ds.mean_phase[f"r{rx_idx}"],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx], label="perfect phi")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(len(ds))],
        label=f"r{rx_idx} gt theta",
    )
    ax[1, rx_idx].plot(trajectory_for_phi(rx_idx, ds), label="EKF")

    ax[0, rx_idx].set_ylabel("radio phi")

    ax[0, rx_idx].legend()
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].legend()
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")
fig.suptitle("Single ladies (radios) EKF")
fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
# paired EKF

offsets = [
    ds.yaml_config["receivers"][0]["theta-in-pis"] * np.pi,
    ds.yaml_config["receivers"][1]["theta-in-pis"] * np.pi,
]


# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def hx_paired(x):
    return np.array(
        [
            np.sin(x[0, 0] - offsets[0]) * antenna_spacing * 2 * np.pi / wavelength,
            np.sin(x[0, 0] - offsets[1]) * antenna_spacing * 2 * np.pi / wavelength,
        ]
    )


def HJacobian_at_paired(x):
    """compute Jacobian of H matrix at x"""
    return np.array(
        [
            [
                np.cos(x[0, 0] - offsets[0]) * antenna_spacing * 2 * np.pi / wavelength,
                0,
            ],
            [
                np.cos(x[0, 0] - offsets[1]) * antenna_spacing * 2 * np.pi / wavelength,
                0,
            ],
        ]
    )

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter

from filterpy.common import Q_discrete_white_noise

from spf.rf import pi_norm, reduce_theta_to_positive_y


def residual(a, b):
    # return pi_norm(a - b)
    y = a - b
    y = y % (2 * np.pi)  # force in range [0, 2 pi)
    if y > np.pi:  # move to [-pi, pi)
        y -= 2 * np.pi
    return y


def residual_paired(a, b):
    # return pi_norm(a - b)
    return np.array([residual(a[0], b[0]), residual(a[1], b[1])])


def trajectory_for_phi_paired(ds):
    rk = ExtendedKalmanFilter(dim_x=2, dim_z=2)
    # initialize with first ground truth state
    y_rad = ds[rx_idx][0]["ground_truth_theta"].item()
    # y_rad_reduced=reduce_theta_to_positive_y(y_rad)
    rk.x = np.array([[y_rad], [0]])

    dt = 0.05
    rk.F = np.eye(2) + np.array([[0, 1], [0, 0]]) * dt

    phi_std = 0.5
    rk.R *= phi_std**2  # can this change dependent on state?
    rk.Q = Q_discrete_white_noise(2, dt=dt, var=1.0)
    rk.P *= 0.1  # initialized as identity?

    traj = []
    for idx in range(len(ds)):
        rk.update(
            np.array(
                [
                    [ds[idx][0]["mean_phase_segmentation"]],
                    [ds[idx][1]["mean_phase_segmentation"]],
                ]
            ),
            HJacobian_at_paired,
            hx_paired,
            residual=residual_paired,
        )
        traj.append(rk.x[0, 0])
        rk.predict()
        rk.x = pi_norm(rk.x)
    return np.array(traj)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import torch_pi_norm

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

for rx_idx in [0, 1]:
    ax[rx_idx].scatter(
        range(ds.mean_phase[f"r{rx_idx}"].shape[0]),
        ds.mean_phase[f"r{rx_idx}"],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[rx_idx].plot(ds.ground_truth_phis[rx_idx], label="perfect phi")

    ax[rx_idx].legend()
    ax[rx_idx].set_xlabel("time step")
    ax[rx_idx].set_ylabel("phi")
    ax[rx_idx].set_title(f"Radio {rx_idx}")

ax[2].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
)
ax[2].set_title("Paired radio EKF")
ax[2].plot(trajectory_for_phi_paired(ds), label="EKF")

ax[2].legend()
fig.suptitle("EKF: When two (radios) become one")
fig.savefig(f"{output_prefix}_when_two_become_one_ekf.png")

In [ ]:
from functools import cache
from filterpy.kalman import ExtendedKalmanFilter

from spf.rf import reduce_theta_to_positive_y, pi_norm


from filterpy.common import Q_discrete_white_noise


@cache
def Q_discrete_white_noise_cached(**kwargs):
    return Q_discrete_white_noise(**kwargs)


@cache
def F_cached(dt):
    return np.eye(2) + np.array([[0, 1], [0, 0]]) * dt


class DualRadioConstantTheta(ExtendedKalmanFilter):

    def __init__(self, phi_std=0.5, p=5, **kwargs):
        super().__init__(**kwargs)
        self.R *= phi_std**2
        self.P[0, 0] *= p  # initialized as identity?
        self.P[1, 1] *= 0.1

    def R_at_x(self, x_for_radio):
        # return 30  # return 1.5
        # return 30 + +2.5 * np.exp(-((abs(x_for_radio) - np.pi / 2) ** 2)) * 30
        return (5 + np.exp(-((abs(x_for_radio) - np.pi / 2) ** 2))) ** 2

    ###
    #    x[0] = theta , ?
    #    x[1] = theta dot , ? / s
    ###
    def predict(self, dt, q_var):
        self.F = F_cached(dt)
        self.Q = Q_discrete_white_noise_cached(
            dim=2, dt=dt, var=q_var
        )  # TODO Cache this
        ### predict self.x
        self.x = np.dot(self.F, self.x)
        self.x[0] = pi_norm(self.x[0])
        ###

        # update covar
        self.P = np.dot(self.F, self.P).dot(self.F.T) + self.Q

        # do some house keeping
        self.x_prior = np.copy(self.x)
        self.P_prior = np.copy(self.P)

    def update(self, z):
        # find a good z!
        r = np.array(
            [
                [self.R_at_x(pi_norm(self.x[0, 0] - offsets[0])), 0],
                [0, self.R_at_x(pi_norm(self.x[0, 0] - offsets[1]))],
            ]
        )
        super().update(
            # np.array([[z]]),
            np.array(z),
            HJacobian_at_paired,
            hx_paired,
            residual=residual_paired,
            R=r,
        )
        # self.x[0] = reduce_theta_to_positive_y(self.x[0])
        self.x[0] = pi_norm(self.x[0])

In [ ]:
def trajectory_for_phi_paired(ds):
    ekf = DualRadioConstantTheta(dim_x=2, dim_z=1, phi_std=0.0, p=5)
    ekf.x = np.array([[ds[rx_idx][0]["ground_truth_theta"].item()], [0]])
    traj = []
    for idx in range(min(25000, len(ds))):
        # ekf.predict(dt=0.1, q_var=0.3) # simulated data
        ekf.predict(dt=0.05, q_var=1.0)  # works for paired
        # ekf.predict(dt=1.0, q_var=0.001)
        ekf.update(
            np.array(
                [
                    [ds[idx][0]["mean_phase_segmentation"]],
                    [ds[idx][1]["mean_phase_segmentation"]],
                ]
            )
        )
        traj.append({"theta": ekf.x[0, 0], "P_theta": ekf.P[0, 0]})
    return traj

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
trajectory = trajectory_for_phi_paired(ds)
n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
# ax[1].plot(
#     [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
#     label=f"r{rx_idx} gt theta",
# )
ax[1].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
)
reduced_gt_theta = np.array(
    [
        reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
        for idx in range(min(n, len(ds)))
    ]
)
ax[1].plot(
    reduced_gt_theta,
    label=f"r{rx_idx} reduced gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
zscores = (xs - reduced_gt_theta) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="EKF-std",
    color="orange",
    alpha=0.2,
)

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio {rx_idx}")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
Q_discrete_white_noise(dim=2, dt=0.05, var=1.0)

In [ ]:
Q_discrete_white_noise(dim=2, dt=1.0, var=0.0001)

In [ ]:
def fx(x, dt):
    F = F_cached(dt)
    return np.dot(F, x)


def hx_paired_ukf(x):
    return np.array(
        [
            np.sin(x[0] - offsets[0]) * antenna_spacing * 2 * np.pi / wavelength,
            np.sin(x[0] - offsets[1]) * antenna_spacing * 2 * np.pi / wavelength,
        ]
    )


def ukf_paired_residual(a, b):
    # return pi_norm(a - b)
    return np.array([residual(a[0], b[0]), residual(a[1], b[1])])


def ukf_paired_residual_x(a, b):
    # return pi_norm(a - b)
    return np.array([pi_norm(a[0] - b[0]), a[0] - b[0]])


def state_mean(sigmas, Wm):
    x = np.zeros(2)
    sum_sin, sum_cos = 0.0, 0.0

    for i in range(len(sigmas)):
        s = sigmas[i]
        x[1] += s[1] * Wm[i]
        sum_sin += np.sin(s[0]) * Wm[i]
        sum_cos += np.cos(s[0]) * Wm[i]
    x[1] /= len(sigmas)
    x[0] = np.arctan2(sum_sin, sum_cos)
    return x

In [ ]:
from filterpy.kalman.sigma_points import MerweScaledSigmaPoints
from filterpy.kalman import UnscentedKalmanFilter

dt = 1.0
points = MerweScaledSigmaPoints(2, alpha=0.1, beta=2.0, kappa=0)  # -1)
kf = UnscentedKalmanFilter(
    dim_x=2,
    dim_z=2,
    dt=dt,
    fx=fx,
    hx=hx_paired_ukf,
    points=points,
    residual_z=ukf_paired_residual,
    # x_mean_fn=state_mean,
    residual_x=ukf_paired_residual_x,
)
kf.x = np.array([0, 0])  # initial state
kf.P[0, 0] *= 5  # initial uncertainty
kf.P[1, 1] *= 0.2  # initial uncertainty
z_std = 5
kf.R = np.diag([z_std**2, z_std**2])  # 1 standard
kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=0.01**2)

trajectory = []

#    for idx in range(min(2500, len(ds))):
for idx in range(min(800, len(ds))):
    print(kf.x)
    kf.predict()
    kf.x[0] = pi_norm(kf.x[0])
    # kf.x[0] = pi_norm(kf.x[0])
    print(kf.x)
    z = np.array(
        [
            ds[idx][0]["mean_phase_segmentation"],
            ds[idx][1]["mean_phase_segmentation"],
        ]
    )
    # print(kf.x)
    kf.update(z)
    kf.x[0] = pi_norm(kf.x[0])
    trajectory.append({"theta": kf.x[0]})  # , "P_theta": ekf.P[0, 0]})
    # print(kf.x, "log-likelihood", kf.log_likelihood)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
# trajectory = trajectory_for_phi_paired(ds)
n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
# ax[1].plot(
#     [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
#     label=f"r{rx_idx} gt theta",
# )
ax[1].plot(
    [torch_pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(ds))],
    label="craft gt theta",
)
reduced_gt_theta = np.array(
    [
        reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
        for idx in range(min(n, len(ds)))
    ]
)
ax[1].plot(
    reduced_gt_theta,
    label=f"r{rx_idx} reduced gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
# stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
# zscores = (xs - reduced_gt_theta) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
# ax[1].fill_between(
#     np.arange(xs.shape[0]),
#     xs - stds,
#     xs + stds,
#     label="EKF-std",
#     color="orange",
#     alpha=0.2,
# )

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio {rx_idx}")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
# lets try a particle filter!

In [ ]:
!pip install pypfilt

In [ ]:
class GaussianWalk(pypfilt.Model):
    def field_types(self, ctx):
        return [("x", np.dtype(float))]

    def update(self, ctx, time_step, is_fs, prev, curr):
        """Perform a single time-step."""
        rnd = ctx.component["random"]["model"]
        step = rnd.normal(loc=0, scale=1, size=curr.shape)
        curr["x"] = prev["x"] + step

In [ ]:
class Lorenz63(OdeModel):
    def field_types(self, ctx):
        r"""
        Define the state vector :math:`[\sigma, \rho, \beta, x, y, z]^T`.
        """
        return [
            ("tehta", float),
            ("tehta_vel", float),
        ]

    def d_dt(self, time, xt, ctx, is_forecast):
        rates = np.zeros(xt.shape, xt.dtype)
        rates["theta"] = xt["tehta_vel"]
        return rates

In [ ]:
np

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset
import numpy as np

from spf.rf import pi_norm

ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
# ds_fn = "/mnt/md0/spf/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"


nthetas = 65
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache="/home/mouse9911/precompute_cache_chunk16",
    paired=True,
    skip_signal_matrix=True,
)

In [ ]:
import pickle

full_p = pickle.load(open("full_p.pkl", "rb"))["full_p"]

In [ ]:
np.random.randn(4, 3)

In [ ]:
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.01
    noise[:, 1] *= 0.001
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights.resize(len(particles))
    weights.fill(1.0 / len(weights))
    weights[1:][change_mask] *= 0.01
    weights /= sum(weights)  # normalize


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, std, dt=1.0):
    N = len(particles)
    particles[:, 0] += dt * particles[:, 1]
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def fix_particles(particles):
    while np.abs(particles[:, 0]).max() > np.pi / 2:
        mask = np.abs(particles[:, 0]) > np.pi / 2
        particles[mask, 0] = np.sign(particles[mask, 0]) * np.pi - particles[mask, 0]
        particles[mask, 1] *= -1
    return particles


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z, R):
    # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
    # print(weights.shape, z.shape, particles.shape)
    # print(z)
    for idx in range(weights.shape[0]):
        weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
        # weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


def pf_trajectory_for_phi(rx_idx, ds):
    N = 128 * 8
    particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.01), N=N)
    weights = np.ones((N,)) / N
    trajectory = []
    thetas = []
    vs = []
    for idx in range(len(ds)):
        particles = fix_particles(particles)
        predict(particles=particles, std=(0.5, 0.1), dt=0.1)
        particles = fix_particles(particles)
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z, R=2)
        particles = fix_particles(particles)
        # print(neff(weights))
        # print(particles.shape)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            # assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))
    return trajectory

In [ ]:
# plt.imshow(
#     np.array(
#         [
#             theta_phi_to_p(np.pi / 4, phi, full_p)
#             for phi in np.linspace(-np.pi, np.pi, 50)
#         ]
#     ).reshape(-1, 1)
# )

In [ ]:
# x = [p[0] for p in thetas]
# y = [p[1] for p in thetas]
# plt.scatter(x, y, s=1.0, alpha=0.5)

In [ ]:
# x = [p[0] for p in vs]
# y = [p[1] for p in vs]
# plt.scatter(x, y, s=1.0, alpha=0.5)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 2, figsize=(10, 15))

for rx_idx in [0, 1]:  # [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    trajectory = pf_trajectory_for_phi(rx_idx, ds)
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
        label=f"r{rx_idx} gt theta",
    )
    reduced_gt_theta = np.array(
        [
            reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
            for idx in range(min(n, len(ds)))
        ]
    )
    ax[1, rx_idx].plot(
        reduced_gt_theta,
        label=f"r{rx_idx} reduced gt theta",
    )

    xs = np.array([x["theta"] for x in trajectory])
    # stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
    # zscores = (xs - reduced_gt_theta) / stds

    ax[1, rx_idx].plot(xs, label="PF-x", color="orange")
    # ax[1, rx_idx].fill_between(
    #     np.arange(xs.shape[0]),
    #     xs - stds,
    #     xs + stds,
    #     label="EKF-std",
    #     color="orange",
    #     alpha=0.2,
    # )

    ax[0, rx_idx].set_ylabel("radio phi")

    ax[0, rx_idx].legend()
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].legend()
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")

    # ax[2, rx_idx].hist(zscores.reshape(-1), bins=25)
fig.suptitle("Single ladies (radios) EKF")
fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
from spf.rf import pi_norm

pi_norm(0.99 * np.pi - (-0.99 * np.pi))

In [ ]:
THIS WORKS

import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.01
    noise[:, 1] *= 0.001
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights[1:][change_mask] *= 0.75
    weights.resize(len(particles))
    weights.fill(1.0 / len(weights))


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, std, dt=1.0):
    N = len(particles)
    particles[:, 0] += dt * particles[:, 1]
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z, R):
    # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
    # print(weights.shape, z.shape, particles.shape)
    # print(z)
    for idx in range(weights.shape[0]):
        weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
        #weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


N = 128 * 8 *8
particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.01), N=N)
weights = np.ones((N,)) / N
trajectory = []
thetas = []
vs = []
for rx_idx in [0]:
    for idx in range(len(ds)):
        particles[:, 0] = pi_norm(particles[:, 0])
        predict(particles=particles, std=(0.5, 0.1), dt=0.1)
        particles[:, 0] = pi_norm(particles[:, 0])
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z, R=2)
        particles[:, 0] = pi_norm(particles[:, 0])
        # print(neff(weights))
        # print(particles.shape)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))

In [ ]:
WORKS2
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]


def resample_from_index(particles, weights, indexes):
    # print(indexes)
    particles[:] = particles[indexes]
    # weights[:] = weights[indexes]
    # add noise to the new samples
    noise = np.random.randn(*particles.shape)
    # noise[:, 0] *= 0.01
    # noise[:, 1] *= 0.01
    noise[:, 0] *= 0.01
    noise[:, 1] *= 0.01
    change_mask = indexes[:-1] == indexes[1:]
    particles[1:][change_mask] += noise[1:][change_mask]
    # weights.resize(len(particles))
    weights.fill(1.0 / len(weights))
    weights[1:][change_mask] *= 0.01
    weights /= sum(weights)  # normalize


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 2))
    particles[:, 0] = mean[0] + (np.random.randn(N) * std[0])
    particles[:, 1] = mean[1] + (np.random.randn(N) * std[1])
    return particles


def predict(particles, std, dt=1.0):
    N = len(particles)
    particles[:, 0] += dt * particles[:, 1]
    # particles[:, 0] += np.random.randn(particles.shape[0]) * 0.0001
    # particles[:, 1] += np.random.randn(particles.shape[0]) * 0.0001


def hx_pf(x):
    return np.sin(x[:, 0]) * antenna_spacing * 2 * np.pi / wavelength


def theta_phi_to_p(theta, phi, full_p):
    theta_bins = full_p.shape[0]
    phi_bins = full_p.shape[1]
    theta_bin = int(theta_bins * (theta + np.pi) / (2 * np.pi)) % theta_bins
    phi_bin = int(phi_bins * (phi + np.pi) / (2 * np.pi)) % phi_bins
    return full_p[theta_bin, phi_bin]


def update(particles, weights, z, R):
    # weights *= scipy.stats.norm(hx_pf(particles), R).pdf(z)
    # print(weights.shape, z.shape, particles.shape)
    # print(z)
    for idx in range(weights.shape[0]):
        weights[idx] *= theta_phi_to_p(particles[idx, 0], z, full_p=full_p)
        # weights[idx] *= scipy.stats.norm(0, 0.0001).pdf(particles[idx, 1])
    weights += 1.0e-300  # avoid round-off to zero
    weights /= sum(weights)  # normalize


def estimate(particles, weights):
    mean = np.average(particles, weights=weights, axis=0)
    var = np.average((particles - mean) ** 2, weights=weights, axis=0)
    return mean, var


def neff(weights):
    return 1.0 / np.sum(np.square(weights))


N = 128 * 8
particles = create_gaussian_particles(mean=np.array([0, 0]), std=(2, 0.01), N=N)
weights = np.ones((N,)) / N
trajectory = []
thetas = []
vs = []
for rx_idx in [0]:
    for idx in range(len(ds)):
        particles[:, 0] = pi_norm(particles[:, 0])
        predict(particles=particles, std=(0.5, 0.1), dt=0.1)
        particles[:, 0] = pi_norm(particles[:, 0])
        z = np.array(ds[idx][rx_idx]["mean_phase_segmentation"])
        update(particles=particles, weights=weights, z=z, R=2)
        particles[:, 0] = pi_norm(particles[:, 0])
        # print(neff(weights))
        # print(particles.shape)
        # resample if too few effective particles
        if neff(weights) < N / 2:
            # print("RESAMPLE")
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            # assert np.allclose(weights, 1 / N)
        mu, var = estimate(particles, weights)
        # particles = create_gaussian_particles(mean=mu, std=np.sqrt(var), N=N)
        trajectory.append({"theta": mu[0]})
        for particle in particles:
            thetas.append((idx, particle[0]))
            vs.append((idx, particle[1]))